#### 把地点转成经纬度

In [ ]:
import json
import requests
import time
# 高德API配置
AMAP_KEY = "d03c7b27db05d88ad9d10d9c64353130"  # 替换成你的Key
GEOCODE_URL = "https://restapi.amap.com/v3/geocode/geo"

def get_geocode(address):
    """调用高德地理编码API获取经纬度"""
    params = {
        "key": AMAP_KEY,
        "address": address,
        "city": "上海市"  # 限定城市提高准确性
    }
    response = requests.get(GEOCODE_URL, params=params)
    data = response.json()
    
    if data["status"] == "1" and data["count"] != "0":
        location = data["geocodes"][0]["location"]  # 格式："经度,纬度"
        lng, lat = location.split(",")
        return {"lng": float(lng), "lat": float(lat)}
    else:
        print(f"地址解析失败: {address}")
        print(f"错误信息: {data['info']}")
        return None

# 读取原始JSON文件
with open("数据\链家租房闵行数据.json", "r", encoding="utf-8") as f:
    rent_data = json.load(f)
success = 0
# 处理每条租房信息
for item in rent_data:
    # 拼接完整地址（示例：上海市徐汇区万体馆银星名庭）
    if item["片区"] != '':
        area_parts = item["片区"].split("-")
        full_address = f"上海市徐汇区{area_parts[1]}{area_parts[2]}"  # 徐汇区-万体馆-银星名庭
    else:
        continue  # 如果片区信息缺失，则跳过该条数据
    
    # 获取经纬度
    location = get_geocode(full_address)
    if location:
        item["location"] = location
        success += 1
    time.sleep(0.8)
# 保存带经纬度的新文件
with open("数据\MinHang_rent_data.json", "w", encoding="utf-8") as f:
    json.dump(rent_data, f, ensure_ascii=False, indent=4)
print(success, "条数据成功添加经纬度信息。")

1431 条数据成功添加经纬度信息。


##### 删除没有地址的数据

In [28]:
import json

# 读取原始JSON文件
with open('数据\MinHang_rent_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)  # 假设文件内容为JSON数组

# 过滤有效数据 (保留有片区信息的数据)
filtered_data= [
    item for item in data 
    if item.get("片区")  # 同时满足：键存在 且 值不为空 且 值非空白字符
]

# 保存清理后的数据
with open('cleaned_data_MinHang.json', 'w', encoding='utf-8') as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=2)

print(f"原始数据量：{len(data)}")
print(f"清理后数据量：{len(filtered_data)}")
print(f"已删除数据量：{len(data) - len(filtered_data)}")

原始数据量：1500
清理后数据量：1431
已删除数据量：69


In [ ]:
import json
AMAP_KEY = "d03c7b27db05d88ad9d10d9c64353130"
with open('cleaned_data_MinHang.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 提取经纬度和房价
points = []
for item in data:
    lng, lat = item['location']['lng'], item['location']['lat']
    price = item['价格(元/月)']
    points.append([float(lng), float(lat), price])  # 格式: [经度, 纬度, 权重]

def generate_heatmap_html(points, api_key):
    html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="utf-8">
        <title>房价热力图</title>
        <script src="https://webapi.amap.com/maps?v=2.0&key={api_key}"></script>
        <script src="https://webapi.amap.com/ui/1.1/main.js"></script>
        <style>
            html, body {{ height: 100%; margin: 0; padding: 0; }}
            #map {{ height: 100%; }}
        </style>
    </head>
    <body>
        <div id="map"></div>
        <script>
            var map = new AMap.Map('map', {{
                zoom: 10,
                center: [{points[0][0]}, {points[0][1]}]
            }});
            
            // 热力图数据
            var heatmapData = {{
                data: {points},
                max: Math.max(...{points}.map(p => p[2]))
            }};
            
            // 创建热力图
            var heatmap = new AMap.Heatmap(map, {{
                radius: 25,  // 热力点半径
                opacity: [0.8, 0.8]  // 透明度
            }});
            heatmap.setDataSet(cleaned_data_MinHang.json);
        </script>
    </body>
    </html>
    """
    with open('heatmap.html', 'w', encoding='utf-8') as f:
        f.write(html)

# 调用函数
generate_heatmap_html(points, AMAP_KEY)

In [32]:
import json

# 读取原始数据
with open('数据\MinHang_rent_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 转换价格并生成热力图专用数据
heatmap_data = []
for item in data:
    try:
        price = int(item["价格(元/月)"])      
        heatmap_data.append({
            "lng": item["location"]["lng"],
            "lat": item["location"]["lat"],
            "value": price  # 热力值
        })
    except:
        print(f"价格转换失败: {item['标题']}")

# 保存热力图专用数据
with open('heatmap_data.json', 'w', encoding='utf-8') as f:
    json.dump(heatmap_data, f, ensure_ascii=False, indent=2)

价格转换失败: 独栋·兆统公寓 闵行店 可爱一居 开间
价格转换失败: 独栋·蜂巢公寓 浦江店 双人间 开间
价格转换失败: 独栋·雅香思寓 江月路店 景观超大一室户 1室1厅
价格转换失败: 独栋·城家公寓 上海新虹桥万象城店 【咨询享86折特价】无中介费！万象城内！近韩国街，爱琴海！ 开间
价格转换失败: 独栋·柚米寓 鑫悦阳光一期 保障房直租 0中介 民用水电 超大阳台 1室1厅
价格转换失败: 独栋·魔方公寓 龙吴路店 华师大附近精装一居室 大户型 独卫 可养猫咪 开间
价格转换失败: 独栋·魔方公寓 龙吴路店 会员专享 年租特惠房 抢手房型，随时入住，可月付， 开间
价格转换失败: 独栋·创客·巢 许浦2店 精装独厨独卫一室2300（临空经济园附近） 开间
价格转换失败: 独栋·创客·巢 许浦2店 精装修南向一室一厅（干净舒适，采光充足，首月租金减半活动） 1室1厅
价格转换失败: 独栋·朗诗寓 莘庄店 空中花园 免费健身房书吧台球厅羽毛球场 近地铁1/5号线 1室1厅
价格转换失败: 独栋·朗诗寓 莘庄店 品牌服务式公寓 精装拎包入住 仲盛世界商城 1室1厅
价格转换失败: 独栋·朗诗寓 莘庄店 年签享折扣，近地铁1/5号线，精装交付，拎包入住 。 1室1厅
价格转换失败: 独栋·城家公寓 上海浦江浦锦路店 近浦江高科技园区！无中介费！大落地窗！带阳台！ 1室1厅
价格转换失败: 独栋·抱家公寓 中春路店 七宝 LOFT空间大可短租 民水民电 1室1厅
价格转换失败: 独栋·城家公寓 联航路地铁站店 免中介费！免费健身房！近地铁，可停车！可以开票! 独厨可做饭 开间
价格转换失败: 独栋·禧玥公寓 联航路店 禧玥公寓,免中介费.联航路地铁500米，独立wifi 1室1厅
价格转换失败: 独栋·城家公寓 联航路地铁站店 免费健身！免费班车！可提公积金！办居住证！民用电！冷水免费！ 开间
价格转换失败: 独栋·禧玥公寓 联航路店 禧玥公寓，免中介费.联航路地铁 500米，独立wifi 1室1厅
价格转换失败: 独栋·禧玥公寓 联航路店 禧玥公寓，联航路地铁500米，无中介费 1室1厅
价格转换失败: 独栋·禧玥公寓 联航路店 禧玥公寓，联航路地铁500米，无中介费用 1室1厅
价格转换失败: 独栋·城家公寓 联航路地铁站店 地铁直达黄浦！免中介费！押一付三!

##### 转换成热力图数据

In [4]:
import json
import math
# 读取原始数据
with open('数据\cleaned_rent_data_Xuhui.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 转换为高德标准格式
heatmap_data = [
    {
        "lng": item["location"]["lng"],
        "lat": item["location"]["lat"],
        "count": math.log(float(item["价格(元/月)"]))  # 注意必须是数值类型
    }
    for item in data
]

# 保存为新的JSON文件
with open('heatmapData_log_Xuhui.json', 'w', encoding='utf-8') as f:
    json.dump(heatmap_data, f, ensure_ascii=False, indent=2)

In [5]:
# 合并数据
import json

def merge_heatmap_data(file1_path, file2_path, output_path):
    """
    合并两个热力图JSON数据文件
    :param file1_path: 第一个JSON文件路径
    :param file2_path: 第二个JSON文件路径
    :param output_path: 合并后输出路径
    """
    try:
        # 1. 读取两个JSON文件
        with open(file1_path, 'r', encoding='utf-8') as f1:
            data1 = json.load(f1)
        with open(file2_path, 'r', encoding='utf-8') as f2:
            data2 = json.load(f2)
        
        # 2. 验证数据格式
        required_keys = {'lng', 'lat', 'count'}
        for data in [data1, data2]:
            if not all(isinstance(item, dict) and required_keys.issubset(item.keys()) for item in data):
                raise ValueError("JSON数据格式不正确，必须包含lng、lat、count字段")
        
        # 3. 合并数据
        merged_data = data1 + data2
        
        # 4. 保存合并后的数据
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(merged_data, f, indent=2, ensure_ascii=False)
        
        print(f"合并成功！共合并 {len(data1)} + {len(data2)} = {len(merged_data)} 条数据")
        print(f"结果已保存到: {output_path}")
    
    except FileNotFoundError as e:
        print(f"文件未找到错误: {e}")
    except json.JSONDecodeError:
        print("错误：JSON文件解析失败，请检查文件格式")
    except ValueError as e:
        print(f"数据格式错误: {e}")

if __name__ == '__main__':
    # 配置输入输出路径
    input_file1 = 'heatmapData_log_Minhang.json'  # 第一个数据文件
    input_file2 = 'heatmapData_log_Xuhui.json'  # 第二个数据文件
    output_file = 'MinHang_Xuhui.json'  # 合并后输出文件
    
    # 执行合并
    merge_heatmap_data(input_file1, input_file2, output_file)

合并成功！共合并 1431 + 1305 = 2736 条数据
结果已保存到: MinHang_Xuhui.json


In [1]:
#数据处理脚本
import json
import math

def process_json(input_path, output_path):
    # 1. 读取原始JSON文件
    with open(input_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 2. 对每个count值取对数
    processed_data = []
    for point in data:
        processed_point = point
        processed_point['count'] = processed_point['count']-6
        processed_data.append(processed_point)
    
    # 3. 保存到新JSON文件
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(processed_data, f, indent=2, ensure_ascii=False)
    
    print(f"处理完成！原始数据已取对数并保存到: {output_path}")

if __name__ == '__main__':
    # 输入输出文件路径（按需修改）
    input_json = 'MinHang_Xuhui.json'    # 原始数据文件
    output_json = '111新尝试.json'  # 处理后保存路径
    
    process_json(input_json, output_json)

处理完成！原始数据已取对数并保存到: 111新尝试.json


In [2]:
# Json to CSV
import json
import pandas as pd

# 读取JSON文件
with open('数据\cleaned_rent_data_Xuhui.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 转换数据结构
processed_data = []
for item in data:
    # 创建一个新字典，包含原始字段
    new_item = {
        "标题": item.get("标题", ""),
        "价格(元/月)": item.get("价格(元/月)", ""),
        "面积": item.get("面积", ""),
        "片区": item.get("片区", ""),
    }
    
    # 提取location中的经纬度
    location = item.get("location", {})
    new_item["lng"] = location.get("lng", "")
    new_item["lat"] = location.get("lat", "")
    
    processed_data.append(new_item)

# 转换为DataFrame
df = pd.DataFrame(processed_data)

# 保存为CSV文件
df.to_csv('数据\houses_Xuhui.csv', index=False, encoding='utf-8-sig')

print(f"转换完成! 共处理 {len(processed_data)} 条数据")
print(f"CSV文件包含以下列: {list(df.columns)}")
print("前5行示例数据:")
print(df.head())

转换完成! 共处理 1305 条数据
CSV文件包含以下列: ['标题', '价格(元/月)', '面积', '片区', 'lng', 'lat']
前5行示例数据:
                 标题 价格(元/月)       面积            片区         lng        lat
0    整租·梅陇十村 1室1厅 南    3800   49.19㎡  徐汇-华东理工-梅陇十村  121.423617  31.130903
1  整租·华利公寓 3室2厅 南/西    9050  124.52㎡    徐汇-田林-华利公寓  121.415203  31.179734
2    整租·银星名庭 1室1厅 南    5200   77.04㎡   徐汇-万体馆-银星名庭  121.437294  31.169061
3    整租·金环大厦 4室3厅 南   28000  225.49㎡  徐汇-建国西路-金环大厦  121.458244  31.204359
4  整租·云锦东方云筑 5室3厅 南  220000  560.00㎡  徐汇-龙华-云锦东方云筑  121.447914  31.170583


In [8]:
import pandas as pd

# 读取主文件（CSV1）和附加数据文件（CSV2）
df_main = pd.read_csv('数据\houses_Minhang.csv')  # 主文件
df_additional = pd.read_csv('数据\Minhang_station_distances.csv')  # 包含附加列的文件

# 检查两个文件的列名
print("主文件列名:", df_main.columns.tolist())
print("附加文件列名:", df_additional.columns.tolist())

# 确定匹配键（使用公共列）
# 假设两个文件都有"house_id"作为匹配键
key = '标题'
# 如果没有house_id，尝试使用位置信息匹配（经度lng和纬度lat）
# elif 'lng' in df_main.columns and 'lat' in df_main.columns and 'house_lon' in df_additional.columns and 'house_lat' in df_additional.columns:
#     # 重命名附加文件的位置列使其一致
#     df_additional = df_additional.rename(columns={
#         'house_lon': 'lng',
#         'house_lat': 'lat'
#     })
#     key = ['lng', 'lat']
# else:
#     print("错误: 无法找到匹配的键(列)")
#     exit()

# 合并数据
df_main['nearest_station_id'] = df_additional['nearest_station_id'] 
df_main['distance_km'] = df_additional['distance_km'] # 初始化新列

# 保存合并后的CSV
output_file = '数据\houses_with_station_info.csv'
df_main.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"文件已保存: {output_file}")

主文件列名: ['标题', '价格(元/月)', '面积', '片区', 'lng', 'lat']
附加文件列名: ['标题', 'nearest_station_id', 'distance_km']
文件已保存: 数据\houses_with_station_info.csv


In [7]:
import pandas as pd
Xuhui = pd.read_csv('XuH_with_StaInfo.csv')
Xuhui['区域'] = '徐汇'
Xuhui['面积(㎡)'] = Xuhui['面积'].str.replace('㎡', '').astype(float)

Minhang = pd.read_csv('MinH_with_StaInfo.csv')
Minhang['区域'] = '闵行'
Minhang['面积(㎡)'] = Minhang['面积'].str.replace('㎡', '').astype(float)

df = pd.concat([Minhang, Xuhui], ignore_index=True)
df['rent_per_unit'] = df['价格(元/月)']/ df['面积(㎡)']
df.drop(columns=['面积'], inplace=True)
df.to_csv('数据\Final_edition_data.csv')

In [3]:
import pandas as pd
df = pd.read_csv('house_data.csv')
df.to_csv('house_data.csv', index=False, encoding='utf-8-sig')

In [4]:
df

,Unnamed: 0,标题,价格(元/月),片区,lng,lat,nearest_station_id,distance_km,station_name,区域,面积(㎡),rent_per_unit
0,0,整租·上海康城 2室1厅 南,4300,闵行-莘闵别墅-上海康城,121.436307,31.188334,1号线,0.6212,上海体育馆,闵行,104.13,41.294536
1,1,整租·长寿新村(闵行) 2室1厅 南,3736,闵行-莘庄南广场-长寿新村(闵行),121.379555,31.105967,5号线,0.7990,莘庄,闵行,67.02,55.744554
2,2,整租·凤庆小区 2室1厅 南,3000,闵行-老闵行-凤庆小区,121.402035,31.013607,5号线,0.8198,金平路,闵行,53.23,56.359196
3,3,整租·中骏广场 1室0厅 南,5000,闵行-华漕-中骏广场,121.314147,31.211750,17号线,2.0180,虹桥火车站,闵行,50.00,100.000000
4,4,整租·鑫都城云天绿洲 1室2厅 南,4000,闵行-颛桥-鑫都城云天绿洲,121.392574,31.052428,5号线,1.8442,颛桥,闵行,58.00,68.965517
...,...,...,...,...,...,...,...,...,...,...,...,...
2731,2731,整租·海珀旭晖酒店式公寓 2室1厅 南/北,18000,徐汇-徐汇滨江-海珀旭晖酒店式公寓,121.458428,31.182624,12号线,0.2340,龙华中路,徐汇,93.14,193.257462
2732,2732,整租·虹桥路400弄 2室2厅 东南/南,11000,徐汇-徐家汇-虹桥路400弄,121.431146,31.192428,11号线,0.6225,徐家汇,徐汇,108.38,101.494741
2733,2733,整租·乌南公馆 4室2厅 南,72000,徐汇-衡山路-乌南公馆,121.449705,31.200852,9号线,0.1642,肇嘉浜路,徐汇,226.53,317.838697
2734,2734,整租·兴国路271弄 1室1厅 南,7500,徐汇-衡山路-兴国路271弄,121.437409,31.205242,11号线,0.3937,交通大学,徐汇,38.55,194.552529
